In [2]:
# Import required libraries
import pandas as pd
import dash
from dash import html as html
from dash import dcc as dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [3]:
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [4]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                    options=[
                                    {'label': 'All Sites', 'value': 'ALL'},
                                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},],
                                    value='ALL',
                                    placeholder="Select a Launch Site here",
                                    searchable=True ),
                                
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),
                                
                                html.P("Payload range (Kg):"),
                                
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0', 10000: '10000'},
                                                value=[min_payload, max_payload]),
                                
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),



                                ])
                                
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
                            
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    if entered_site == 'ALL':
       filtered_df = spacex_df[['Launch Site','class']]
       filtered_df = filtered_df .groupby(['Launch Site'], as_index=False).sum()
       fig = px.pie(filtered_df, values=filtered_df['class'], names=filtered_df['Launch Site'], title='Total Success Launches By Site')
       return fig   
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df[spacex_df["Launch Site"] == entered_site]
        filtered_df = filtered_df[['class']].value_counts().reset_index().rename(columns={0: "total"})
        fig = px.pie(filtered_df, values=filtered_df['total'], names=filtered_df['class'], 
                     title='Total Success Launches fo Site {}'.format(entered_site))                                                   
        return fig                   
                                
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output                                
                                
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),Input(component_id='payload-slider', component_property='value')])

def get_scatter_chart(entered_site, entered_payload):
    if entered_site == 'ALL':
       fig = px.scatter(spacex_df, x =spacex_df['Payload Mass (kg)'], y =spacex_df['class'], color= spacex_df['Booster Version Category'], 
                 title='Correlation between succes rate and payload mass for all sites')
       return fig   
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df[spacex_df["Launch Site"] == entered_site]
        fig = px.scatter(filtered_df, x= filtered_df['Payload Mass (kg)'], y = filtered_df['class'],
                     color= filtered_df['Booster Version Category'], 
                     title='Correlation between succes rate and payload mass site {}'.format(entered_site))                                                   
        return fig                                 
                                
                                
                                
                                
                                
                                                  
                            
                                

if __name__ == '__main__':
    app.run_server()